In [2]:
from xml.dom import minidom
from unidecode import unidecode
import sys
import pandas as pd
import numpy as np
import json
sys.path.insert(1, '../utils')
from read_config import read_config_file

In [3]:
inputs_path = '../inputs/'
results_path = '../results/'

## Read TXT config file

In [4]:
config_dict = {}
read_config_file(inputs_path + 'INDEX.CFG', config_dict)
config_dict

{'LEIA': ['lista.csv'], 'ESCREVA': ['modelo.json']}

## Read GLI

In [5]:
lista_df = pd.read_csv(results_path + config_dict['LEIA'][0], sep=';')
lista_df.head()

,WORDS,RECORDS
0,the,"['00001', '00001', '00001', '00001', '00001', ..."
1,significance,"['00001', '00074', '00078', '00121', '00147', ..."
2,of,"['00001', '00001', '00001', '00001', '00001', ..."
3,pseudomonas,"['00001', '00001', '00001', '00007', '00008', ..."
4,aeruginosa,"['00001', '00001', '00001', '00006', '00006', ..."


In [6]:
filtered_lista_df = lista_df[
    (lista_df['WORDS'].str.len() >= 2) &
    (lista_df['WORDS'].str.isalpha()) 
]


#### TF-IDF paramters

- N = number of documents
- $n_j$ = number of documents with word j
- $tf_{ij}$= frequency of j in i


In [7]:
freq_dict = {}
word_doc_dict = {}
docs_max_freq = {}
for index, row in filtered_lista_df.iterrows():
    word = row['WORDS']
    records = eval(row['RECORDS'])
    
    word_doc_dict[word] = len(set(records))
    for doc in records:
        if doc in  docs_max_freq:
            docs_max_freq[doc] = max(docs_max_freq[doc], records.count(doc))
        else:
            docs_max_freq[doc] = records.count(doc)
        freq_dict.setdefault(word, {})[doc] = records.count(doc)

## Geneta Matrix with tf-idf as weights

In [8]:
def tf_idf_default(word_doc_dict, freq_dict, docs_max_freq):
    words_list = word_doc_dict.keys()
    docs_list = docs_max_freq.keys()
    N = len(docs_list)
    tf_matrix = {}
    for doc in docs_list:
        for word in words_list:
            if doc in freq_dict[word]:
                tf_matrix.setdefault(word, {})[doc] = (freq_dict[word][doc]/docs_max_freq[doc]) * np.log(N/word_doc_dict[word])
            else:
                tf_matrix.setdefault(word, {})[doc] = 0 
    return tf_matrix

In [12]:
tf_matrix = tf_idf_default(word_doc_dict, freq_dict, docs_max_freq)
tf_matrix_df = pd.DataFrame(tf_matrix)

In [71]:
tf_matrix_df.to_csv(results_path + config_dict['ESCREVA'][0], sep=';')